In [1]:
!pip3 install onnx

In [2]:
import sys
import torch.onnx
import onnx
import cv2
from vision.ssd.mobilenetv3_ssd_lite import create_mobilenetv3_small_ssd_lite
#from caffe2.python.onnx.backend import Caffe2Backend as c2

In [3]:
model_path = "./models/mbv3-v2-Epoch-65-Loss-4.263951171823099.pth"
label_path = "./models/open-images-model-labels.txt"

In [4]:
class_names = [name.strip() for name in open(label_path).readlines()]
num_classes = len(class_names)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
net = create_mobilenetv3_small_ssd_lite(len(class_names), is_test=True)
net.load(model_path)
net = net.to(DEVICE)

In [6]:
net.eval()

SSD(
  (base_net): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): hswish()
    (3): Block(
      (se): SeModule(
        (se): Sequential(
          (0): AdaptiveAvgPool2d(output_size=1)
          (1): Conv2d(16, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): ReLU(inplace=True)
          (4): Conv2d(4, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (6): hsigmoid()
        )
      )
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nolinear1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_

In [7]:
#宣言
net_type = "mbv3-ssd-v1_fix"
model_path = f"models/{net_type}.onnx"
init_net_path = f"models/{net_type}_init_net.pb"
init_net_txt_path = f"models/{net_type}_init_net.pbtxt"
predict_net_path = f"models/{net_type}_predict_net.pb"
predict_net_txt_path = f"models/{net_type}_predict_net.pbtxt"

In [8]:
dummy_input = torch.randn(1, 3, 300, 300)
torch.onnx.export(net, dummy_input, model_path, verbose=True, output_names=['scores', 'boxes'])

graph(%input.1 : Float(1, 3, 300, 300, strides=[270000, 90000, 300, 1], requires_grad=0, device=cpu),
      %classification_headers.0.3.weight : Float(24, 48, 1, 1, strides=[48, 1, 1, 1], requires_grad=1, device=cpu),
      %classification_headers.0.3.bias : Float(24, strides=[1], requires_grad=1, device=cpu),
      %classification_headers.1.3.weight : Float(24, 576, 1, 1, strides=[576, 1, 1, 1], requires_grad=1, device=cpu),
      %classification_headers.1.3.bias : Float(24, strides=[1], requires_grad=1, device=cpu),
      %classification_headers.2.3.weight : Float(24, 512, 1, 1, strides=[512, 1, 1, 1], requires_grad=1, device=cpu),
      %classification_headers.2.3.bias : Float(24, strides=[1], requires_grad=1, device=cpu),
      %classification_headers.3.3.weight : Float(24, 256, 1, 1, strides=[256, 1, 1, 1], requires_grad=1, device=cpu),
      %classification_headers.3.3.bias : Float(24, strides=[1], requires_grad=1, device=cpu),
      %classification_headers.4.3.weight : Float(24,

In [9]:
model = onnx.load(model_path)
onnx.save(model, model_path)

In [ ]:
#next C:\Program Files (x86)\IntelSWTools\openvino_2020.3.355\deployment_tools\model_optimizer optimize